<a href="https://colab.research.google.com/github/AchmadLutfi196/23059-datamining/blob/main/Analisis_Sentimen_Ulasan_Tiktok_Dengan_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
alvianardiansyah_dataset_ulasan_pengguna_shopee_path = kagglehub.dataset_download('alvianardiansyah/dataset-ulasan-pengguna-shopee')

print('Data source import complete.')


# Analisis Sentimen Pengguna Aplikasi Shopee di Google Play Store Terkait Fitur COD dengan Menggunakan Metode SVM****

Review pada Google Play Store merupakan aspek penting yang digunakan pengguna untuk mengekspresikan penilaian mereka terhadap sebuah aplikasi. Aplikasi Shopee adalah platform e-commerce yang memungkinkan pengguna untuk melakukan berbagai aktivitas, seperti berbelanja, menjual produk, dan berinteraksi dengan penjual maupun pembeli lainnya. Aplikasi ini menyediakan fitur-fitur seperti penawaran menarik, sistem pembayaran yang mudah, layanan pelanggan, dan berbagai promosi.

Dalam sebuah penelitian, dilakukan proses pengumpulan data dari ulasan pengguna aplikasi Shopee melalui tahap scraping. Selanjutnya, data tersebut dilakukan pelabelan dengan mengkategorikan ulasan ke dalam kelas positif, negatif, dan netral. Kemudian, dilakukan text preprocessing untuk mengolah data dengan membersihkan, menyeleksi, dan mengubahnya menjadi data yang lebih terstruktur agar dapat diolah lebih lanjut sesuai kebutuhan penelitian.

Setelah mendapatkan data hasil text preprocessing, langkah berikutnya adalah melakukan pembobotan kata menggunakan metode Term Frequency – Inverse Document Frequency (TF-IDF). Metode ini memberikan nilai pada setiap kata dalam sebuah dokumen berdasarkan frekuensi kemunculan kata tersebut dalam seluruh dokumen yang ada.

Selanjutnya, data yang telah diolah menjalani proses klasifikasi menggunakan algoritma Support Vector Machine (SVM). SVM digunakan untuk mengklasifikasikan ulasan menjadi kategori sentimen yang tepat, seperti positif, negatif, atau netral berdasarkan pola-pola yang teridentifikasi dari data ulasan yang telah diproses sebelumnya.

Manfaat dilakukan analisis sentimen adalah:

1. Hasil analisis sentimen dapat dilihat oleh pengguna aplikasi Shopee dan digunakan untuk memahami opini pengguna lain.

2. Sebagai bahan pertimbangan bagi Shopee untuk meningkatkan dan menyempurnakan platformnya berdasarkan hasil analisis sentimen yang diperoleh.

    Dengan mengubah aplikasi yang disebutkan dari MyXL menjadi Shopee, fokusnya berpindah dari platform layanan telekomunikasi ke platform e-commerce. Dalam konteks ini, analisis sentimen tetap relevan untuk memahami opini pengguna dan memperbaiki layanan yang disediakan oleh Shopee.

# 1. Scraping Data

Scraping data dilakukan dengan mengumpukan data ulasan pengguna aplikasi Shopee dari Google Play Store menggunakan bantuan dari library google-play-scraper dan IDE Google Collab. Data yang diambil berupa nama pemberi ulasan, nilai bintang yang diberikan, waktu ulasan dikirim dan isi review ulasan.

In [ ]:
!pip install google-play-scraper

In [ ]:
from google_play_scraper import app
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews

In [ ]:
result, continuation_token = reviews(
    'com.shopee.id',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=50000,
    filter_score_with=None
)
df_shopee = pd.DataFrame(np.array(result), columns=['review'])
df_shopee = df_shopee.join(pd.DataFrame(df_shopee.pop('review').tolist()))
df_shopee_cod = df_shopee[df_shopee['content'].str.contains('COD', case=False)]

In [ ]:
df_shopee_cod.head()

In [ ]:
new_df = df_shopee_cod[['userName', 'score','at', 'content']]
sorted_df = new_df.sort_values(by='at', ascending=False)
sorted_df.to_csv("Data ulasan Shopee tentang COD.csv", index = False)

# 2. Case Folding

Case folding dilakukan pengubahan seluruh huruf menjadi kecil (lowercase) yang ada pada dokumen. Tahap ini akan dibantu dengan bantuan library RegEx.

In [ ]:
import re

def cleaningulasan(content):
    ulasan = re.sub(r'@[A-Za-a0-9]+',' ', content)
    ulasan = re.sub(r'#[A-Za-z0-9]+',' ', content)
    ulasan = re.sub(r"http\S+",' ', content)
    ulasan = re.sub(r'[0-9]+',' ', content)
    ulasan = re.sub(r"[-()\"#/@;:<>{}'+=~|.!?,_]", " ", content)
    ulasan = content.strip(' ')
    return ulasan

def clearEmoji(content):
    return content.encode('ascii', 'ignore').decode('ascii')

def replaceTOM(content):
    pola = re.compile(r'(.)\1{2,}', re.DOTALL)
    return pola.sub(r'\1', content)

def casefoldingText(content):
    ulasan = content.lower()
    return ulasan

# Mendefinisikan DataFrame baru dengan kolom yang dipilih
new_df_copy = new_df.copy()

# Membersihkan konten menggunakan fungsi yang telah dibuat
new_df_copy['Cleaning'] = new_df_copy['content'].apply(cleaningulasan)
new_df_copy['HapusEmoji'] = new_df_copy['Cleaning'].apply(clearEmoji)
new_df_copy['3/Lebih'] = new_df_copy['HapusEmoji'].apply(replaceTOM)
new_df_copy['CaseFolding'] = new_df_copy['3/Lebih'].apply(casefoldingText)

# Mengurutkan DataFrame berdasarkan kolom 'at'
sorted_df = new_df_copy.sort_values(by='at', ascending=False)

# Menyimpan DataFrame yang telah diolah ke dalam file CSV
sorted_df.to_csv("Data ulasan Shopee tentang COD.csv", index=False)


In [ ]:
new_df_copy.head()

# 3. Pelabelan Dataset

Pelabelan dataset dilakukan secara manual terhadap pelabelan data kedalam kelas kategori positif, negatif atau netral. Proses ini akan dilakukan dengan bantuan ahli bahasa Indonesia dan dilakukan pelabelan oleh setidaknya lebih dari dua orang.

In [ ]:
label = []
for index, row in new_df.iterrows():
  if row['score'] == 1 or row['score'] == 2:
    label.append('Negatif')
  elif row['score'] == 3:
    label.append('Netral')
  else:
    label.append('Positif')
new_df_copy['sentimen'] = label

In [ ]:
new_df_copy.head()

# 4. Tokenizing

Hasil proses case folding, kalimat akan diproses dengan menguraikannya menjadi token-token atau kata-kata. Pada tahap ini akan dibantu dengan library NLTK.

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenizingText(content):
    ulasan = word_tokenize(content)
    return ulasan

# Menerapkan tokenizingText pada setiap elemen di kolom 'CaseFolding'
new_df_copy['Tokenizing'] = new_df_copy['CaseFolding'].apply(tokenizingText)

In [ ]:
new_df_copy.head()

# 5. Formalisasi

Tahap formalisasi dilakukan untuk pengubah penggunaan kata tidak baku menjadi baku sesuai dengan KBBI. Proses akan menggunakan file dataset slangwords yang berisi kata slang yang nanti akan diubah menjadi baku. Tahap ini dibantu dengan library RegEx.

In [ ]:
def convertToSlangword(tokens):
    # Load slang word dictionary from a file
    with open("/kaggle/input/d/alvianardiansyah/dataset-ulasan-pengguna-shopee/slangwords.txt", "r") as file:
        kamusSlang = eval(file.read())

    pattern = re.compile(r'\b(' + '|'.join(kamusSlang.keys()) + r')\b')
    content = []

    for kata in tokens:
        filterSlang = pattern.sub(lambda x: kamusSlang[x.group()], kata)
        content.append(filterSlang.lower())

    return content

# Assuming 'new_df' contains the data
new_df_copy['Formalisasi'] = new_df_copy['Tokenizing'].apply(convertToSlangword)

In [ ]:
new_df_copy.head(20)

# 6. Stopword Removal

Hasil proses formalisasi kemudian akan dilakukan seleksi kata yang tidak penting dan menghapus kata tersebut. Tahap ini dibantu dengan library NLTK.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

daftar_stopword = stopwords.words('indonesian')
# Append additional stopwords
daftar_stopword.extend(["yg", "dg", "rt"])
daftar_stopword = set(daftar_stopword)

def stopwordText(words):
    return [word for word in words if word not in daftar_stopword]

# Assuming 'new_df' contains the data and 'Tokenizing' column has the tokenized words
new_df_copy['WithoutStopwords'] = new_df_copy['Tokenizing'].apply(stopwordText)

In [ ]:
new_df_copy.head()

# 7. Stemming

Proses stemming dilakukan perubahan kata yang berimbuhan menjadi kata dasar. Tahap ini dibantu dengan library Sastrawi dan Swifter.

In [ ]:
!pip install Sastrawi
!pip install swifter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in new_df_copy['WithoutStopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

def stemmingText(document):
    return [term_dict[term] for term in document]

new_df_copy['Stemming'] = new_df_copy['WithoutStopwords'].swifter.apply(stemmingText)

In [ ]:
new_df_copy.head()

# 8. Visualisasi Data (NLP)
Visualisasi data dalam pemrosesan bahasa alami (Natural Language Processing/NLP) melibatkan representasi grafis dari informasi yang terdapat dalam teks atau data teks yang telah diproses. Ini membantu dalam pemahaman, analisis, dan komunikasi hasil dari algoritma pemrosesan bahasa alami.

In [ ]:
# Jika new_df_copy adalah salinan yang ingin dimodifikasi
train_s0 = new_df_copy[new_df_copy['sentimen'] == 'Negatif'].copy()

# Mengisi nilai yang hilang pada kolom 'content' untuk baris yang terfilter
train_s0.loc[:, 'content'] = train_s0['content'].fillna("tidak ada komentar")

# Menampilkan beberapa baris pertama dari DataFrame yang telah dimodifikasi
train_s0.head()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
all_text_s0 = ' '.join(word for word in train_s0['content'])
wordcloud = WordCloud(colormap='Reds', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s0)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)
plt.title("Sentimen Negatif")
plt.show()

In [ ]:
train_s1 = new_df_copy[new_df_copy['sentimen'] == 'Netral'].copy()
train_s1.loc[:, 'content'] = train_s1['content'].fillna("tidak ada komentar")
train_s1.head()

In [ ]:
all_text_s1 = ' '.join(word for word in train_s1['content'])
wordcloud = WordCloud(colormap='Blues', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s1)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)
plt.title("Sentimen Netral")
plt.show()

In [ ]:
train_s2 = new_df_copy[new_df_copy['sentimen'] == 'Positif'].copy()
train_s2.loc[:, 'content'] = train_s2['content'].fillna("tidak ada komentar")
train_s2.head()

In [ ]:
all_text_s2 = ' '.join(word for word in train_s2['content'])
wordcloud = WordCloud(colormap='Greens', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s2)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)
plt.title("Sentimen Positif")
plt.show()

In [ ]:
sentimen_data = pd.value_counts(new_df_copy['sentimen'], sort=True)
sentimen_data.plot(kind='bar', color=['red', 'green', 'lightskyblue'])
plt.title("Bar Chart")
plt.show

# 9. Pembobotan Kata dengan TF-IDF

Pada tahap ini dilakukan pembobotan kata dari hasil stemming dengan metode Term Inverse Document Frequency (TF-IDF).Metode TF-IDF digunakan untuk mengetahui seberapa sering suatu kata muncul di dalam dokumen. Tahap ini dibantu dengan library sklearn.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Pastikan new_df berisi kolom 'Stemming' dan 'sentimen'
X = new_df_copy['Stemming']
Y = new_df_copy['sentimen']

# Misalkan x_train dan x_test adalah list token yang ingin digabungkan kembali menjadi teks
x_text = [' '.join(tokens) for tokens in X]

# Sekarang x_text berisi teks yang bisa digunakan untuk fit_transform
x_train, x_test, y_train, y_test = train_test_split(x_text, Y, test_size=0.2)

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.transform(y_test)


In [ ]:
# x_train dan x_test telah di-vektorisasi menggunakan TfidfVectorizer
# vectorizer = TfidfVectorizer()
# x_train = vectorizer.fit_transform(x_train)
# x_test = vectorizer.transform(x_test)

# Hitung total kemunculan setiap kata di semua dokumen
word_freq = x_train.sum(axis=0)

# Dapatkan indeks dari fitur kata pada vektorisasi
feature_names = vectorizer.get_feature_names_out()

# Buat daftar kata beserta jumlah kemunculannya
word_freq_list = [(word, word_freq[0, idx]) for word, idx in zip(feature_names, range(len(feature_names)))]

# Urutkan berdasarkan jumlah kemunculan kata
word_freq_list = sorted(word_freq_list, key=lambda x: x[1], reverse=True)

# Tampilkan beberapa kata beserta jumlah kemunculannya
num_words_to_display = 10  # Ganti dengan jumlah kata yang ingin ditampilkan
for word, freq in word_freq_list[:num_words_to_display]:
    print(f"Kata: {word}, Kemunculan: {freq}")


# 10. Klasifikasi dengan SVM

Klasifikasi ulasan pengguna dilakukan menggunakan algoritma Support Vector Machine yang akan dibantu dengan library Scikit-Learn. Proses klasifikasi menggunakan nilai data latih dan data uji sebesar 80%:20%, dilakukan percobaan sebanyak 5x dan kernel akan menggunakan kernel rbf dan kernel linear.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

SVM = svm.SVC(kernel='rbf')    #Jika dengan Kernel RBF
SVM.fit(x_train,y_train)

acc_score_rbf = cross_val_score(SVM, x_train, y_train, cv=5, scoring='accuracy')
pre_score_rbf = cross_val_score(SVM, x_train, y_train, cv=5, scoring='precision_macro')
rec_score_rbf = cross_val_score(SVM, x_train, y_train, cv=5, scoring='recall_macro')
f_score_rbf = cross_val_score(SVM, x_train, y_train, cv=5, scoring='f1_macro')

print('Hasil Accuracy : %s' % (acc_score_rbf))
print('Hasil Rata - Rata Accuracy : %s' % acc_score_rbf.mean())
print('Hasil Precision : %s' % (pre_score_rbf))
print('Hasil Rata - Rata Precision : %s' % pre_score_rbf.mean())
print('Hasil Recall : %s' % (rec_score_rbf))
print('Hasil Rata - Rata Recall : %s' % rec_score_rbf.mean())
print('Hasil F-Measure : %s' % (f_score_rbf))
print('Hasil Rata - Rata F-Measure : %s' % f_score_rbf.mean())

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

SVM = svm.SVC(kernel='linear') #Jika dengan Kernel Linear
SVM.fit(x_train,y_train)

acc_score_linear = cross_val_score(SVM, x_train, y_train, cv=5, scoring='accuracy')
pre_score_linear = cross_val_score(SVM, x_train, y_train, cv=5, scoring='precision_macro')
rec_score_linear = cross_val_score(SVM, x_train, y_train, cv=5, scoring='recall_macro')
f_score_linear = cross_val_score(SVM, x_train, y_train, cv=5, scoring='f1_macro')

print('Hasil Accuracy : %s' % (acc_score_linear))
print('Hasil Rata - Rata Accuracy : %s' % acc_score_linear.mean())
print('Hasil Precision : %s' % (pre_score_linear))
print('Hasil Rata - Rata Precision : %s' % pre_score_linear.mean())
print('Hasil Recall : %s' % (rec_score_linear))
print('Hasil Rata - Rata Recall : %s' % rec_score_linear.mean())
print('Hasil F-Measure : %s' % (f_score_linear))
print('Hasil Rata - Rata F-Measure : %s' % f_score_linear.mean())

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Data evaluasi yang ingin diplot
scores = {
    'Accuracy': acc_score_rbf.mean(),
    'Precision': pre_score_rbf.mean(),
    'Recall': rec_score_rbf.mean(),
    'F-Measure': f_score_rbf.mean()
}

# Membuat DataFrame dari data evaluasi
eval_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Score'])

# Plot grafik garis Linear
eval_df['Score'].plot(kind='line', marker='o', color='blue')  # Grafik garis dengan penanda titik biru
plt.title('Hasil Evaluasi Model RBF')
plt.xlabel('Metrik')
plt.ylabel('Nilai Rata-Rata')
plt.grid(True)  # Menambahkan grid
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Data evaluasi yang ingin diplot
scores = {
    'Accuracy': acc_score_linear.mean(),
    'Precision': pre_score_linear.mean(),
    'Recall': rec_score_linear.mean(),
    'F-Measure': f_score_linear.mean()
}

# Membuat DataFrame dari data evaluasi
eval_df = pd.DataFrame.from_dict(scores, orient='index', columns=['Score'])

# Plot grafik garis Linear
eval_df['Score'].plot(kind='line', marker='o', color='blue')  # Grafik garis dengan penanda titik biru
plt.title('Hasil Evaluasi Model Linear')
plt.xlabel('Metrik')
plt.ylabel('Nilai Rata-Rata')
plt.grid(True)  # Menambahkan grid
plt.show()


# Hasil Klasifikasi

Hasil klasifikasi didapatkan bahwa pada pengujian pertama penggunaan data utuh berjumlah 1840 data didapatkan bahwa hasil akurasi terbaik pada data utuh dengan pengujian kernel Linear menghasilkan nilai akurasi sebesar 72,37%. Sedangkan untuk pengujian dengan kernel RBF tidak jauh berbeda hanya saja masih di bawah dari kernel linear yaitu sebesar 71,89%.

Dapat disimpulkan bahwa penggunaan kernel yang digunakan akan sangat berpengaruh terhadap hasil akurasi maka dari perlu untuk mengguji dengan berbagai kernel agar dapat mengetahui kernel terbaik.